<a href="https://colab.research.google.com/github/seyonechithrananda/ncov-ligand-protein/blob/master/Cross_fold_Stratified_GAT_ncov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we experiment with a Single Task Stratified Splitter (instead of a ScaffoldSplitter) as it balances each training batch with both positive and negative samples. Given our dataset suffers from imbalanced of hits to negative samples for Mpro inhibition, this may help the model learn a better representation. Despite this notion, the notebook doesn't achieve any advantage over a scaffold based training split.

In [1]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')


--2020-05-07 04:05:34--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  81.12M   155MB/s    in 0.5s    

2020-05-07 04:05:34 (155 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
  

In [2]:
!conda install -c dglteam dgl-cuda10.1

Solving environment: - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dgl-cuda10.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |         openblas          46 KB
    certifi-2020.4.5.1         |           py37_0         155 KB
    decorator-4.4.2            |             py_0          14 KB
    dgl-cuda10.1-0.4.3post2    |           py37_0        11.2 MB  dglteam
    networkx-2.4               |             py_0         1.2 MB
    scipy-1.4.1                |   py37habc2bb6_0        14.6 MB
    ------------------------------------------------------------
                                           Total:        27.1 MB

The following NEW packages will be INSTALLED:

  blas               pkgs/main/linux-64::blas-1.0-openblas
  decorator          pkgs/main/

In [3]:
!conda install -c dglteam dgllife


Solving environment: - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dgllife


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dgllife-0.2.1              |           py37_0         132 KB  dglteam
    joblib-0.14.1              |             py_0         201 KB
    scikit-learn-0.22.1        |   py37h22eb022_0         5.3 MB
    ------------------------------------------------------------
                                           Total:         5.6 MB

The following NEW packages will be INSTALLED:

  dgllife            dglteam/linux-64::dgllife-0.2.1-py37_0
  joblib             pkgs/main/noarch::joblib-0.14.1-py_0
  scikit-learn       pkgs/main/linux-64::scikit-learn-0.22.1-py37h22eb022_0


Proceed ([y]/n)? y


joblib-0.14.1        | 201 KB    | : 100% 1.0/1 [00:00<00:00,  7.88it

In [4]:
!conda install pandas

Solving environment: / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |           py37_0         2.8 MB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    pandas-1.0.3               |   py37h0573a6f_0         8.6 MB
    ------------------------------------------------------------
                                           Total:        13.9 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.1.1-0
  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main:

In [0]:
import os 
import sys 
import pandas as pd

# train --> balanced dataset
dataset_train_file = "/content/drive/My Drive/Project De Novo/AID1706_binarized_sars_full_eval_actives_12k_samples.csv"
dataset_eval_file = "/content/drive/My Drive/Project De Novo/mpro_xchem.csv"
dataset_train = pd.read_csv(dataset_train_file)
dataset_eval = pd.read_csv(dataset_eval_file)

In [6]:
dataset_train.head

<bound method NDFrame.head of                                                   smiles  activity
0      C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)...         1
1      CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C...         1
2      CC(C)(C)NC(=O)C(C1=CSC=C1)N(C2=CC=C(C=C2)N)C(=...         1
3      CC(C)C(=O)NC1=CC=C(C=C1)N(C(C2=CSC=C2)C(=O)NC(...         1
4      CC(C)C(=O)NC1=CC=C(C=C1)N(CC2=CSC=C2)C(=O)CN3C...         1
...                                                  ...       ...
11994                               C1=CC2=C(C=C1N)NN=C2         0
11995  CC(=O)[C@H]1CC[C@@H]2[C@@]1(CC(=O)[C@H]3[C@H]2...         0
11996                       C1CN(CCN1CC(CO)O)C2=CC=CC=C2         0
11997  CCOC(=O)N1CCC(=C2C3=C(CCC4=C2N=CC=C4)C=C(C=C3)...         0
11998                    C1=CC2=C(C=C1OC(F)(F)F)SC(=N2)N         0

[11999 rows x 2 columns]>

In [7]:
dataset_eval.head

<bound method NDFrame.head of                                 smiles  activity
0      OC=1C=CC=CC1CNC2=NC=3C=CC=CC3N2         1
1        CC(=O)NCCC1=CNC=2C=CC(F)=CC12         1
2    O=C([C@@H]1[C@H](C2=CSC=C2)CCC1)N         1
3       CN1CCCC=2C=CC(=CC12)S(=O)(=O)N         1
4     CC(=O)NC=1C=CC(OC=2N=CC=CN2)=CC1         1
..                                 ...       ...
875   CC(C)C=1C=CC(NC(=O)N2CCOCC2)=CC1         0
876        CN(CC(=O)O)C(=O)C=1C=CC=CN1         0
877  CN1CCN(CC1)C(=O)C=2C=CC(F)=C(F)C2         0
878      FC=1C=CC=C(F)C1C(=O)N2CCCCCC2         0
879             FC=1C=CC=NC1NCC2CCOCC2         0

[880 rows x 2 columns]>

In [8]:
from dgllife.data import MoleculeCSVDataset
from dgllife.data.csv_dataset import *
from dgllife.utils.featurizers import *
from dgllife.utils.mol_to_graph import *

# featurize bigraph/molecular graph set for train (SARS-COV-1) set
train_set = MoleculeCSVDataset(dataset_train, smiles_to_graph=smiles_to_bigraph, node_featurizer=CanonicalAtomFeaturizer(),
                               edge_featurizer=CanonicalBondFeaturizer(), smiles_column='smiles', cache_file_path='/content/drive/My Drive/Project De Novo/graph_featurizer/train.bin', task_names=['activity'])

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Loading previously saved dgl graphs...


In [9]:
# featurize bigraph/molecular graph set for test (SARS-COV-2) set
test_set = MoleculeCSVDataset(dataset_eval, smiles_to_graph=smiles_to_bigraph, node_featurizer=CanonicalAtomFeaturizer(),
                               edge_featurizer=CanonicalBondFeaturizer(), smiles_column='smiles', cache_file_path='/content/drive/My Drive/Project De Novo/graph_featurizer/test.bin', task_names=['activity'])

Loading previously saved dgl graphs...


In [10]:
from google.colab import files
files.upload()

Saving utils.py to utils.py


{'utils.py': b'import dgl\nimport numpy as np\nimport random\nimport torch\n\nfrom dgllife.utils.featurizers import one_hot_encoding\nfrom dgllife.utils.splitters import RandomSplitter\n\ndef set_random_seed(seed=0):\n    """Set random seed.\n    Parameters\n    ----------\n    seed : int\n        Random seed to use\n    """\n    random.seed(seed)\n    np.random.seed(seed)\n    torch.manual_seed(seed)\n    if torch.cuda.is_available():\n        torch.cuda.manual_seed(seed)\n\n\ndef load_dataset_for_classification(args):\n    """Load dataset for classification tasks.\n    Parameters\n    ----------\n    args : dict\n        Configurations.\n    Returns\n    -------\n    dataset\n        The whole dataset.\n    train_set\n        Subset for training.\n    val_set\n        Subset for validation.\n    test_set\n        Subset for test.\n    """\n    assert args[\'dataset\'] in [\'Tox21\']\n    if args[\'dataset\'] == \'Tox21\':\n        from dgllife.data import Tox21\n        dataset = Tox

In [0]:
args = {
    'random_seed': 2,
    'batch_size': 128,
    'lr': 1e-3,
    'num_epochs': 250,
    'node_data_field': 'h',
    'frac_train': 0.8,
    'frac_val': 0.1,
    'frac_test': 0.1,
    'in_feats': 74,
    'gat_hidden_feats': [32, 32],
    'classifier_hidden_feats': 64,
    'num_heads': [4, 4],
    'patience': 50,
    'smiles_to_graph': smiles_to_bigraph,
    'node_featurizer': CanonicalAtomFeaturizer(),
    'metric_name': 'roc_auc_score',
    'model': 'GAT'
}


In [0]:
import numpy as np
import torch
import utils

from dgllife.model import load_pretrained
from dgllife.utils import EarlyStopping, Meter
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader


from utils import set_random_seed, load_dataset_for_classification, collate_molgraphs, load_model

from dgllife.model import GATPredictor

args['device'] = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_random_seed(args['random_seed'])



In [18]:
print(train_set.task_names)

['activity']


In [0]:
from dgllife.utils.splitters import SingleTaskStratifiedSplitter

train_stratified_set, val_set, test_stratified_set = SingleTaskStratifiedSplitter.train_val_test_split(train_set, 
                                                                      frac_train=0.8, frac_val=0.2,frac_test=0.0, labels=train_set.labels, task_id=0)

In [20]:
test_fold_set = SingleTaskStratifiedSplitter.k_fold_split(test_set, labels=test_set.labels, task_id=0, k=5, log=True)

Processing fold 1/5
Processing fold 2/5
Processing fold 3/5
Processing fold 4/5
Processing fold 5/5


In [21]:
print(type(test_fold_set[4][1]))

<class 'dgl.data.utils.Subset'>


In [83]:
'''
print (len(train_set))
print(len(train_stratified_set))
print((train_stratified_set[0]))
print (len(val_set))
print (len(test_set))
print(train_set[1])
'''

# misformatted graph, removed manually.

from dgl.data.utils import Subset

print(train_stratified_set[1][1].ndata)

print(train_stratified_set[9600])

valid_train_stratified_set = []


for idx, graph in enumerate(train_stratified_set):
      if idx==9600:
        print(train_stratified_set[9600])
        break
      valid_train_stratified_set.append(graph)

train_stratified_set=valid_train_stratified_set

'''
for idx, graph in enumerate(train_stratified_set):
  print (idx)
  print (train_stratified_set[idx][1].ndata)
  if train_stratified_set[idx][1].ndata is None:
    print (idx)
    train_stratified_set.remove(idx)

'''

{'h': tensor([[1., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])}
('C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)C(=O)NCC4=CC=CC=C4)C(=O)CN5C6=CC=CC=C6N=N5', DGLGraph(num_nodes=44, num_edges=98,
         ndata_schemes={}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}), tensor([1.]), tensor([1.]))
('C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)C(=O)NCC4=CC=CC=C4)C(=O)CN5C6=CC=CC=C6N=N5', DGLGraph(num_nodes=44, num_edges=98,
         ndata_schemes={}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}), tensor([1.]), tensor([1.]))


'\nfor idx, graph in enumerate(train_stratified_set):\n  print (idx)\n  print (train_stratified_set[idx][1].ndata)\n  if train_stratified_set[idx][1].ndata is None:\n    print (idx)\n    train_stratified_set.remove(idx)\n\n'

In [84]:
print(len(train_stratified_set))
print(type(train_set))
print(type(train_stratified_set))
print(train_stratified_set[0])

9600
<class 'dgllife.data.csv_dataset.MoleculeCSVDataset'>
<class 'list'>
('CN1C=CC=C1C(C(=O)NC2CCCCC2)N(C3=CC4=C(C=C3)OCO4)C(=O)CC5=CC=CS5', DGLGraph(num_nodes=34, num_edges=76,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}), tensor([0.]), tensor([1.]))


In [0]:
train_loader = DataLoader(train_stratified_set,  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader = DataLoader(val_set,  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)

test_loader = DataLoader(test_set,  batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)

In [0]:
test_fold_loader_1 = DataLoader(test_fold_set[0][0], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)

test_fold_loader_2 = DataLoader(test_fold_set[0][1], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_3 = DataLoader(test_fold_set[1][0], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_4 = DataLoader(test_fold_set[1][1], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_5 = DataLoader(test_fold_set[2][0], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_6 = DataLoader(test_fold_set[2][1], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_7 = DataLoader(test_fold_set[3][0], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_8 = DataLoader(test_fold_set[3][1], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_9 = DataLoader(test_fold_set[4][0], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)
test_fold_loader_10 = DataLoader(test_fold_set[4][1], batch_size=args['batch_size'],
                          collate_fn=collate_molgraphs)


In [87]:
print(len(test_loader))
print(len(test_fold_loader_1))
print(len(test_fold_loader_2))
print(len(test_fold_loader_3))
print(len(test_fold_loader_4))
print(len(test_fold_loader_5))


7
6
2
6
2
6


In [101]:
args['n_tasks'] = 1
model = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

import dgl.backend as F

train_num_pos = F.sum(train_set.labels, dim=0)
train_num_indices = F.sum(train_set.mask, dim=0)
train_task_pos_weights = (train_num_indices - train_num_pos) / train_num_pos


loss_criterion = BCEWithLogitsLoss(pos_weight=train_task_pos_weights.to(args['device']),
                                    reduction='none')


#loss_criterion = BCEWithLogitsLoss(reduction='none')

optimizer = Adam(model.parameters(), lr=args['lr'])
stopper = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/train_stratified.pth')
model.to(args['device'])


GATPredictor(
  (gnn): GAT(
    (gnn_layers): ModuleList(
      (0): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=74, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=74, out_features=128, bias=False)
        )
      )
      (1): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=128, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
  )
  (readout): WeightedSumAndMax(
    (weight_and_sum): WeightAndSum(
      (atom_weighting): Sequential(
        (0): Linear(in_features=32, out_features=1, bias=True)
        (1): Sigmoid()


In [102]:
print(train_task_pos_weights)

tensor([25.9036])


In [0]:

def predict(args, model, bg):
  
    node_feats = bg.ndata.pop(args['node_data_field']).to(args['device'])
    if args.get('edge_featurizer', None) is not None:
        edge_feats = bg.edata.pop(args['edge_data_field']).to(args['device'])
        return model(bg, node_feats, edge_feats)
    else:
        return model(bg, node_feats)

def run_a_train_epoch(args, epoch, model, data_loader, loss_criterion, optimizer):
    model.train()
    train_meter = Meter()
    for batch_id, batch_data in enumerate(data_loader):
        smiles, bg, labels, masks = batch_data
        labels, masks = labels.to(args['device']), masks.to(args['device'])
        print(bg)
        logits = predict(args, model, bg)
        # Mask non-existing labels
        loss = (loss_criterion(logits, labels) * (masks != 0).float()).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('epoch {:d}/{:d}, batch {:d}/{:d}, loss {:.4f}'.format(
            epoch + 1, args['num_epochs'], batch_id + 1, len(data_loader), loss.item()))
        train_meter.update(logits, labels, masks)
    train_score = np.mean(train_meter.compute_metric(args['metric_name']))
    print('epoch {:d}/{:d}, training {} {:.4f}'.format(
        epoch + 1, args['num_epochs'], args['metric_name'], train_score))

def run_an_eval_epoch(args, model, data_loader):
    model.eval()
    eval_meter = Meter()
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            labels = labels.to(args['device'])
            logits = predict(args, model, bg)
            eval_meter.update(logits, labels, masks)
    return np.mean(eval_meter.compute_metric(args['metric_name']))


In [104]:
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model, train_loader, loss_criterion, optimizer)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model, val_loader)
        early_stop = stopper.step(val_score, model)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper.best_score))
        if early_stop:
            break

Streaming output truncated to the last 5000 lines.
DGLGraph(num_nodes=3286, num_edges=7118,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 39/75, loss 0.6732
DGLGraph(num_nodes=3223, num_edges=7018,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 40/75, loss 0.6758
DGLGraph(num_nodes=3256, num_edges=7048,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 41/75, loss 0.6724
DGLGraph(num_nodes=3241, num_edges=7046,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 44/250, batch 42/75, loss 0.6778
DGLGraph(num_nodes=3270, num_edges=7092,
         ndata_s

In [105]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_loader)
print('test {} {:.4f}'.format(args['metric_name'], test_score))


test roc_auc_score 0.6928


In [106]:
print(type(test_fold_loader_1))
print(type(test_fold_loader_2))
print(type(test_fold_loader_3))
print(type(test_fold_loader_4))
print(type(test_fold_loader_5))

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader.DataLoader'>


In [107]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_1)
print('test_fold_1 {} {:.4f}'.format(args['metric_name'], test_score))


test_fold_1 roc_auc_score 0.6926


In [0]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_2)
print('test_fold_2 {} {:.4f}'.format(args['metric_name'], test_score))


In [108]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_3)
print('test_fold_3 {} {:.4f}'.format(args['metric_name'], test_score))


test_fold_3 roc_auc_score 0.6966


In [0]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_4)
print('test_fold_4 {} {:.4f}'.format(args['metric_name'], test_score))


In [109]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_5)
print('test_fold_5 {} {:.4f}'.format(args['metric_name'], test_score))


test_fold_5 roc_auc_score 0.6887


In [0]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_6)
print('test_fold_6 {} {:.4f}'.format(args['metric_name'], test_score))


In [110]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_7)
print('test_fold_7 {} {:.4f}'.format(args['metric_name'], test_score))


test_fold_7 roc_auc_score 0.6948


In [0]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_8)
print('test_fold_8 {} {:.4f}'.format(args['metric_name'], test_score))


In [0]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_9)
print('test_fold_9 {} {:.4f}'.format(args['metric_name'], test_score))


In [111]:
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_fold_loader_10)
print('test_fold_10 {} {:.4f}'.format(args['metric_name'], test_score))


test_fold_10 roc_auc_score 0.7019
